In [1]:
import os
import gc
import sys
import copy
import glob
import tqdm

%load_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from torch import nn
import random
import torch
import platform
from typing import Callable, List, Optional, Dict
import numpy as np
import scipy.sparse as sp

import warnings
warnings.filterwarnings('ignore')

from transformers import AutoTokenizer, AutoModel

import torch_geometric
from torch_geometric.data import (
    Data,
    InMemoryDataset,
    Batch
    )
import torch_geometric.datasets as datasets
import torch_geometric.transforms as transforms
from torch_geometric.data import Data
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool, global_max_pool

# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, silhouette_score

from unixcoder import UniXcoder

# To ensure determinism
seed = 1234
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed)

# Check versions
print(torch.__version__)
print(torch.version.cuda)
print(platform.python_version())
print(torch_geometric.__version__)

2.2.1
11.8
3.12.2
2.5.0


In [3]:
from collections import OrderedDict

def get_nodes_edges(inTextFile, add_reverse_edges = False, api_name = None):
  # FD = 0, CD = 1
  # to support the hetero data object as suggested by the documentation 
  nodes_dict = OrderedDict()
  edge_indices_CD = []
  edge_indices_FD = []

  #to support the Data object as used by the Entities dat object as used in RGAT source code
  edge_indices = []
  edge_type = []
  
  # nodes_dict is an index_map
  node_count=0
  fp = open(inTextFile, "r")
    
  file_name = inTextFile.split("/")[-1].strip()
  Lines = fp.readlines()
  
  # Capture the API nodes first
  number_of_api_nodes = 0
  if api_name != None:
    api_name = "." + api_name.split(".")[-1].strip() + "("
    #api_name = api_name[api_name.find("."):] + "("
    for line in Lines:
      nodes = line.split('-->')
      nodes[0], nodes[1] = nodes[0].strip(), nodes[1].strip()
      
      src = nodes[0]  
      if src not in nodes_dict.keys() and api_name in src:
        nodes_dict[src] = node_count
        node_count += 1
        number_of_api_nodes += 1
        
      right_idx = nodes[1].rfind('[')
      dst = nodes[1][:right_idx].strip()
      if dst not in nodes_dict.keys() and api_name in dst:
        nodes_dict[dst] = node_count
        node_count += 1
        number_of_api_nodes += 1
    if number_of_api_nodes == 0:
      print("No API Nodes found!!!!")
    
  # Process each edge
  for line in Lines:

      N = line.split('-->')
      N[0], N[1] = N[0].strip(), N[1].strip()
      
      #t1 = N[0].split('$$')   
      src = N[0].strip()   
      if src not in nodes_dict.keys():
        nodes_dict[src] = node_count
        node_count+=1
        
      #t2 = N[1].split('$$')
      right_idx = N[1].rfind('[')
      dst = N[1][:right_idx].strip()
      if dst not in nodes_dict.keys():
        nodes_dict[dst] = node_count
        node_count+=1

      x = N[1].strip()[right_idx + 1 : -1].strip()
      if(x == 'FD'):
        y=0
        edge_type.append(y)
        edge_indices.append([nodes_dict[src], nodes_dict[dst]])
        if add_reverse_edges:
          edge_type.append(y)
          edge_indices.append([nodes_dict[dst], nodes_dict[src]])
        edge_indices_FD.append([nodes_dict[src], nodes_dict[dst]])
      else: 
        y=1
        edge_type.append(y)
        edge_indices.append([nodes_dict[src], nodes_dict[dst]])
        if add_reverse_edges:
          edge_type.append(y)
          edge_indices.append([nodes_dict[dst], nodes_dict[src]])
        edge_indices_CD.append([nodes_dict[src], nodes_dict[dst]])
     
  return nodes_dict, edge_indices_FD, edge_indices_CD, edge_indices, edge_type, file_name, number_of_api_nodes

In [2]:
from parser import DFG_python,DFG_java,DFG_ruby,DFG_go,DFG_php,DFG_javascript
from parser import (remove_comments_and_docstrings,
                   tree_to_token_index,
                   index_to_code_token,
                   tree_to_variable_index)
from tree_sitter import Language, Parser

dfg_function={
    'python':DFG_python,
    'java':DFG_java,
    'ruby':DFG_ruby,
    'go':DFG_go,
    'php':DFG_php,
    'javascript':DFG_javascript
}

from torch.utils.data import DataLoader, Dataset

from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                  RobertaConfig, RobertaModel, RobertaTokenizer)

# Set GPU
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

#build model
tokenizer = RobertaTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = RobertaModel.from_pretrained("microsoft/graphcodebert-base").to(device)

#load parsers
parsers={}
for lang in dfg_function:
    LANGUAGE = Language('parser/my-languages.so', lang)
    parser = Parser()
    parser.set_language(LANGUAGE) 
    parser = [parser,dfg_function[lang]]
    parsers[lang]= parser

#remove comments, tokenize code and extract dataflow
def extract_dataflow(code, parser, lang):
    #remove comments
    try:
        code = remove_comments_and_docstrings(code,lang)
    except:
        pass    
    #obtain dataflow
    if lang=="php":
        code="<?php"+code+"?>"    
    try:
        tree = parser[0].parse(bytes(code,'utf8'))    
        root_node = tree.root_node  
        tokens_index=tree_to_token_index(root_node)     
        code=code.split('\n')
        code_tokens=[index_to_code_token(x,code) for x in tokens_index]  
        index_to_code={}
        for idx,(index,code) in enumerate(zip(tokens_index,code_tokens)):
            index_to_code[index]=(idx,code)  
        try:
            DFG,_=parser[1](root_node,index_to_code,{}) 
        except:
            DFG=[]
        DFG=sorted(DFG,key=lambda x:x[1])
        indexs=set()
        for d in DFG:
            if len(d[-1])!=0:
                indexs.add(d[1])
            for x in d[-1]:
                indexs.add(x)
        new_DFG=[]
        for d in DFG:
            if d[1] in indexs:
                new_DFG.append(d)
        dfg=new_DFG
    except:
        dfg=[]
    return code_tokens,dfg

class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 code_tokens,
                 code_ids,
                 position_idx,
                 dfg_to_code,
                 dfg_to_dfg
    ):
        self.code_tokens = code_tokens
        self.code_ids = code_ids
        self.position_idx=position_idx
        self.dfg_to_code=dfg_to_code
        self.dfg_to_dfg=dfg_to_dfg

def convert_examples_to_features(item):
    func, tokenizer = item
    parser = parsers['java']
    code_length = 256
    data_flow_length = 64
    
    #extract data flow
    code_tokens,dfg=extract_dataflow(func, parser, 'java')
    code_tokens=[tokenizer.tokenize('@ '+x)[1:] if idx!=0 else tokenizer.tokenize(x) for idx,x in enumerate(code_tokens)]
    ori2cur_pos={}
    ori2cur_pos[-1]=(0,0)
    for i in range(len(code_tokens)):
        ori2cur_pos[i]=(ori2cur_pos[i-1][1],ori2cur_pos[i-1][1]+len(code_tokens[i]))    
    code_tokens=[y for x in code_tokens for y in x]
    
    #truncating
    code_tokens=code_tokens[:code_length+data_flow_length-2-min(len(dfg),data_flow_length)]
    code_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
    code_ids =  tokenizer.convert_tokens_to_ids(code_tokens)
    position_idx = [i+tokenizer.pad_token_id + 1 for i in range(len(code_tokens))]
    dfg=dfg[:code_length+data_flow_length-len(code_tokens)]
    code_tokens+=[x[0] for x in dfg]
    position_idx+=[0 for x in dfg]
    code_ids+=[tokenizer.unk_token_id for x in dfg]
    padding_length=code_length+data_flow_length-len(code_ids)
    position_idx+=[tokenizer.pad_token_id]*padding_length
    code_ids+=[tokenizer.pad_token_id]*padding_length    
    #reindex
    reverse_index={}
    for idx,x in enumerate(dfg):
        reverse_index[x[1]]=idx
    for idx,x in enumerate(dfg):
        dfg[idx]=x[:-1]+([reverse_index[i] for i in x[-1] if i in reverse_index],)
    dfg_to_dfg=[x[-1] for x in dfg]
    dfg_to_code=[ori2cur_pos[x[1]] for x in dfg]
    length=len([tokenizer.cls_token])
    dfg_to_code=[(x[0]+length,x[1]+length) for x in dfg_to_code]
    
    return InputFeatures(code_tokens, code_ids, position_idx, dfg_to_code, dfg_to_dfg)

class TextDataset(Dataset):
    def __init__(self, func, tokenizer):
        self.examples = []
        data = [(func, tokenizer)]
        self.examples = list(map(convert_examples_to_features, data))
        
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item): 
        code_length = 256
        data_flow_length = 64
    
        #calculate graph-guided masked function
        attn_mask=np.zeros((code_length+data_flow_length,
                            code_length+data_flow_length),dtype=bool)
        #calculate begin index of node and max length of input
        node_index=sum([i>1 for i in self.examples[item].position_idx])
        max_length=sum([i!=1 for i in self.examples[item].position_idx])
        #sequence can attend to sequence
        attn_mask[:node_index,:node_index]=True
        #special tokens attend to all tokens
        for idx,i in enumerate(self.examples[item].code_ids):
            if i in [0,2]:
                attn_mask[idx,:max_length]=True
        #nodes attend to code tokens that are identified from
        for idx,(a,b) in enumerate(self.examples[item].dfg_to_code):
            if a<node_index and b<node_index:
                attn_mask[idx+node_index,a:b]=True
                attn_mask[a:b,idx+node_index]=True
        #nodes attend to adjacent nodes 
        for idx,nodes in enumerate(self.examples[item].dfg_to_dfg):
            for a in nodes:
                if a+node_index<len(self.examples[item].position_idx):
                    attn_mask[idx+node_index,a+node_index]=True  
                    
        return (torch.tensor(self.examples[item].code_ids),
              torch.tensor(attn_mask),
              torch.tensor(self.examples[item].position_idx))

def get_embedding(code_inputs=None, attn_mask=None, position_idx=None): 
    nodes_mask=position_idx.eq(0)
    token_mask=position_idx.ge(2)
    inputs_embeddings = model.embeddings.word_embeddings(code_inputs)
    nodes_to_token_mask = nodes_mask[:,:,None]&token_mask[:,None,:]&attn_mask
    nodes_to_token_mask = nodes_to_token_mask/(nodes_to_token_mask.sum(-1)+1e-10)[:,:,None]
    avg_embeddings = torch.einsum("abc,acd->abd",nodes_to_token_mask,inputs_embeddings)
    inputs_embeddings = inputs_embeddings*(~nodes_mask)[:,:,None]+avg_embeddings*nodes_mask[:,:,None]
    return model(inputs_embeds=inputs_embeddings,attention_mask=attn_mask,position_ids=position_idx)[1]

def get_code_embeddings_from_graphcodebert(codelines):
    gc.collect()
    torch.cuda.empty_cache()
    code = "\n".join(codelines)
    dataset = TextDataset(code, tokenizer)
    dataloader = DataLoader(dataset, batch_size=1)
    for batch in enumerate(dataloader):
        code_inputs = batch[1][0].to(device)
        attn_mask = batch[1][1].to(device)
        position_idx = batch[1][2].to(device)
        embedding = get_embedding(code_inputs, attn_mask, position_idx)
    return torch.flatten(embedding)

Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def get_embeddings_from_llms(folders, model, filter_small_examples = False):
  embeddings = {}
  MINIMUM_LENGTH_TO_CONSIDER = 5
  for label, folder in tqdm.tqdm(enumerate(folders)):
    folder_name = folder.strip().split("/")[-1]
    print("\nProcessing: {}\n".format(folder_name))
    embeddings[folder_name] = []
    files = glob.glob(os.path.join(folder, '*.java'))
    print("\nNumber of files: {}\n".format(len(files)))
    random.shuffle(files)
    count = 0
    for file in files:
      if(count == 1500):
         break
      sample_name = file.split("/")[-2].strip()
      file_name = file.split("/")[-1].strip()
      if(count % 5 == 0):
          print("\nAt file: {}\n".format(count))
                        
      fp = open(file,'r')
      lines = fp.readlines()
      lines = [line for line in lines if not line.startswith("import") and not len(line.strip('\n')) == 0]
      lines = [line.strip('\n').strip("\t").strip(" ") for line in lines]
      
      if(filter_small_examples and len(lines) < MINIMUM_LENGTH_TO_CONSIDER):
        continue
      
      if model == "codebert":
        embedding = get_code_embeddings_from_codebert(lines)
      elif model == "unixcoder":
        embedding = get_code_embeddings_from_unixcoder(lines)
      elif model == "codeT5+":
        embedding = get_code_embeddings_from_codet5p(lines)
      elif model == "graphcodebert":
        embedding = get_code_embeddings_from_graphcodebert(lines)
      embedding = embedding.detach().cpu().numpy()
      embeddings[folder_name].append([sample_name + "_" + file_name, sample_name, embedding])
      count += 1
    
  return embeddings

In [ ]:
OUTPUT_FOLDER_LOCATION = "./API-Minsuse/Repository/Benchmarks/Code2Seq-Examples/code2seq_processed_java_data_mubench"
project_folders = [os.path.join(OUTPUT_FOLDER_LOCATION, name) for name in os.listdir(OUTPUT_FOLDER_LOCATION) if os.path.isdir(os.path.join(OUTPUT_FOLDER_LOCATION, name))]

embeddings = get_embeddings_from_llms(project_folders, "graphcodebert")

0it [00:00, ?it/s]


Processing: java.nio.ByteBuffer.get


Number of files: 727


At file: 0


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25


At file: 30


At file: 35


At file: 40


At file: 45


At file: 50


At file: 55


At file: 60


At file: 65


At file: 70


At file: 75


At file: 80


At file: 85


At file: 90


At file: 95


At file: 100


At file: 105


At file: 110


At file: 115


At file: 120


At file: 125


At file: 130


At file: 135


At file: 140


At file: 145


At file: 150


At file: 155


At file: 160


At file: 165


At file: 170


At file: 175


At file: 180


At file: 185


At file: 190


At file: 195


At file: 200


At file: 205


At file: 210


At file: 215


At file: 220


At file: 225


At file: 230


At file: 235


At file: 240


At file: 245


At file: 250


At file: 255


At file: 260


At file: 265


At file: 270


At file: 275


At file: 280


At file: 285


At file: 290


At file: 295


At file: 300


At file: 305


At file: 310


At file: 315




1it [01:29, 89.08s/it]


Processing: java.util.ArrayList.remove


Number of files: 525


At file: 0


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25


At file: 30


At file: 35


At file: 40


At file: 45


At file: 50


At file: 55


At file: 60


At file: 65


At file: 70


At file: 75


At file: 80


At file: 85


At file: 90


At file: 95


At file: 100


At file: 105


At file: 110


At file: 115


At file: 120


At file: 125


At file: 130


At file: 135


At file: 140


At file: 145


At file: 150


At file: 155


At file: 160


At file: 165


At file: 170


At file: 175


At file: 180


At file: 185


At file: 190


At file: 195


At file: 200


At file: 205


At file: 210


At file: 215


At file: 220


At file: 225


At file: 230


At file: 235


At file: 240


At file: 245


At file: 250


At file: 255


At file: 260


At file: 265


At file: 270


At file: 275


At file: 280


At file: 285


At file: 290


At file: 295


At file: 300


At file: 305


At file: 310


At file: 315

2it [02:33, 74.46s/it]

...

37it [1:05:24, 108.48s/it]


Processing: java.sql.PreparedStatement.executeUpdate


Number of files: 301


At file: 0


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25


At file: 30


At file: 35


At file: 40


At file: 45


At file: 50


At file: 55


At file: 60


At file: 65


At file: 70


At file: 75


At file: 80


At file: 85


At file: 90


At file: 95


At file: 100


At file: 105


At file: 110


At file: 115


At file: 120


At file: 125


At file: 130


At file: 135


At file: 140


At file: 145


At file: 150


At file: 155


At file: 160


At file: 165


At file: 170


At file: 175


At file: 180


At file: 185


At file: 190


At file: 195


At file: 200


At file: 205


At file: 210


At file: 215


At file: 220


At file: 225


At file: 230


At file: 235


At file: 240


At file: 245


At file: 250


At file: 255


At file: 260


At file: 265


At file: 270


At file: 275


At file: 280


At file: 285


At file: 290


At file: 295



38it [1:06:10, 89.94s/it] 


At file: 300


Processing: javax.crypto.Mac.init


Number of files: 34


At file: 0


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25


At file: 30



39it [1:06:16, 101.95s/it]


In [11]:
import clustering_and_evaluation 
import importlib
importlib.reload(clustering_and_evaluation)

total_sh, total_db, total_ch, total_ri, total_mu = 0, 0, 0, 0, 0
project_count = 0
cluster_count_list = []

for folder in project_folders:
    folder_name = folder.strip().split("/")[-1]
    print("\nAnalyzing: ", folder_name)
    only_file_names, only_embeddings, ground_truth = [], [], []
    label_mapping = {}
    label = 0
    if folder_name not in embeddings or len(embeddings[folder_name]) == 1:
        print(f"\n{folder_name} API is not found in Code2Seq Data or have only one example!!\n")
        continue
    for file_name, sample_name, embedding in embeddings[folder_name]:
        if sample_name in label_mapping:
            ground_truth.append(label_mapping[sample_name])
        else:
            label_mapping[sample_name] = label
            label += 1
            ground_truth.append(label_mapping[sample_name])
        only_embeddings.append(embedding)
        only_file_names.append(file_name)
        
    print("Total no of examples: {} and no of ground-truth clusters: {}\n".format(len(only_file_names), len(set(ground_truth))))
    silhouette_result, davies_bouldin_result, calinski_harabasz_result, adjusted_rand_result, fowlkes_mallows_result, adjusted_mutual_info_result, v_measure_result, avg_cluster_count, cluster_count  = clustering_and_evaluation.find_and_evaluate_best_clustering(only_embeddings, only_file_names, ground_truth, "Birch", "DB")
    cluster_count_list.append([folder_name, avg_cluster_count, cluster_count])
    if silhouette_result != None and davies_bouldin_result != None:
        project_count += 1
        total_sh += silhouette_result
        total_db += davies_bouldin_result
        total_ri += adjusted_rand_result
        total_mu += adjusted_mutual_info_result
    print("===========================================================================================")
    
print("\nAverage SH score: {} and DB score: {}".format(round(total_sh/project_count, 3), round(total_db/project_count, 3)))
print("Average RI score: {} and MI score: {}".format(round(total_ri/project_count, 3), round(total_mu/project_count, 3)))


Analyzing:  java.nio.ByteBuffer.get
Total no of examples: 727 and no of ground-truth clusters: 1

Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Best silhouette score: 0.10000000149011612 for cluster count: 3
Best davies_bouldin score: 1.3 for cluster count: 32
Best calinski_harabasz score: 77.079 for cluster count: 3
Average best cluster number: 32 using DB score

Clustering for cluster number:  32
Cluster Counts:  {0: 457, 6: 72, 2: 67, 21: 40, 13: 28, 18: 16, 1: 8, 12: 6, 26: 3, 4: 3, 20: 2, 23: 2, 25: 2, 15: 2, 24: 2, 31: 1, 19: 1, 30: 1, 22: 1, 16: 1, 29: 1, 9: 1, 10: 1, 17: 1, 14: 1, 27: 1, 7: 1, 11: 1, 8: 1, 28: 1, 3: 1, 5: 1}
Cluster Mapping:  {0: ['java.nio.ByteBuffer.get_getString.java', 'java.nio.ByteBuffer.get_getInt16.java', 'java.nio.ByteBuffer.get_parseBooleanArray.java', 'java.nio.ByteBuffer.get_calcCrc32.java', 'java.nio.ByteBuffer.get_byteAt.java', 'java.nio.ByteBuffer.get_readRuntimeParameterAnnotations.java', 'java.nio.B

In [ ]:
OUTPUT_FOLDER_LOCATION = "./API-Minsuse/Repository/Benchmarks/MuBench/after_processing"
project_folders = [os.path.join(OUTPUT_FOLDER_LOCATION, name) for name in os.listdir(OUTPUT_FOLDER_LOCATION) if os.path.isdir(os.path.join(OUTPUT_FOLDER_LOCATION, name))]

llm_gcb_embeddings = get_embeddings_from_llms(project_folders, "graphcodebert")

0it [00:00, ?it/s]


Processing: Line.toSubSpace


Number of files: 4


At file: 0



1it [00:00,  1.28it/s]


Processing: Number.doubleValue


Number of files: 4


At file: 0



2it [00:01,  1.36it/s]

...

79it [00:45,  1.89it/s]


Processing: ItemState.getStatus


Number of files: 2


At file: 0



80it [00:46,  2.12it/s]


Processing: File.createNewFile


Number of files: 2


At file: 0



81it [00:46,  1.74it/s]


In [18]:
api_names = []
file_names = []
labels = []
num_data_points = []
max_examples = []
ground_truth_labels = []
minimum_required_points = [[],[],[],[],[],[],[],[]]
predicted_labels = [[],[],[],[],[],[],[],[]]

THRESHOLDS = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4] # 0.2 for 20% of largest cluster

def find_clusters_for_llm_data(api_name, cluster_no, cluster_count):
    only_file_names, only_embeddings, ground_truth = [], [], []
    label_mapping = {}
    label = 0
    for file_name, sample_name, embedding in embeddings[api_name]:
        #print(sample_name)
        if sample_name in label_mapping:
            ground_truth.append(label_mapping[sample_name])
        else:
            label_mapping[sample_name] = label
            label += 1
            ground_truth.append(label_mapping[sample_name])
        only_embeddings.append(embedding)
        only_file_names.append(file_name)
    
    print("\nTotal datapoints: {} and no of clusters: {}\n".format(len(only_embeddings), cluster_no))
    birch_model = Birch(n_clusters = cluster_no, threshold = 3.0)
    birch_model.fit(only_embeddings)

    mubench_api_name = None
    for name in llm_gcb_embeddings:
        if api_name.endswith("." + name):
            mubench_api_name = name
            break
    else:
        print("MuBench API name not found!!")
        return
    
    for file_name, sample_name, embedding in llm_gcb_embeddings[mubench_api_name]:
        cluster_label = birch_model.predict(embedding.reshape(1, -1))
        print("For eaxample: {}, cluster label: {}".format(file_name, cluster_label[0]))
        
        api_names.append(api_name)
        file_names.append(file_name)
        labels.append(cluster_label[0])
        num_data_points.append(cluster_count[cluster_label[0]])
        
        # Get the first cluster label and count as a single variable in the format "label: count"
        # first_label, first_count = next(iter(cluster_count.items()))
        # first_cluster = f"{first_label}: {first_count}"
        # max_examples.append(first_cluster)
        
        # Predict label
        ground_truth_label = int(file_name.strip().split("_")[-1].split(".")[0])
        ground_truth_labels.append(ground_truth_label)
        total_data_points = 0
        for e in cluster_count:
            total_data_points += cluster_count[e]
        max_examples.append(total_data_points)
        
        for i in range(0, len(THRESHOLDS)):
            minimum_required_point = int(total_data_points * THRESHOLDS[i])
            if num_data_points[-1] >= minimum_required_point:
                predicted_label = 0
            else:
                predicted_label = 1
            minimum_required_points[i].append(minimum_required_point)
            predicted_labels[i].append(predicted_label)

In [19]:
# api_list = [["java.io.BufferedReader.readLine", 63],
#            ["java.lang.ClassLoader.loadClass", 32],
#            ["java.lang.Thread.start", 465],
#            ["java.net.URL.openConnection", 16],
#            ["java.sql.DriverManager.getConnection", 10],
#            ["java.sql.Statement.execute", 14],
#            ["java.lang.String.charAt", 268],
#            ["java.lang.String.getBytes", 296],
#            ["java.lang.String.substring", 406],
#            ["java.io.PrintWriter.write", 3],
#            ["java.security.MessageDigest.getInstance", 2]]

for api_name, cluster_no, cluster_count in cluster_count_list:
    print("\nProcessing: {}".format(api_name))
    find_clusters_for_llm_data(api_name, cluster_no, cluster_count)


Processing: java.nio.ByteBuffer.get

Total datapoints: 727 and no of clusters: 32

For eaxample: ByteBuffer.get_FlipBuffer_0.java, cluster label: 0
For eaxample: ByteBuffer.get_wrapBuffer_0.java, cluster label: 6
For eaxample: ByteBuffer.get_FlipBuffer_1.java, cluster label: 0
For eaxample: ByteBuffer.get_wrapBuffer_1.java, cluster label: 6

Processing: java.util.ArrayList.remove

Total datapoints: 525 and no of clusters: 124

For eaxample: ArrayList.remove_CheckMarkersForNull_0.java, cluster label: 2
For eaxample: ArrayList.remove_CheckMarkersForNull_1.java, cluster label: 5

Processing: java.util.stream.IntStream.range

Total datapoints: 43 and no of clusters: 3

For eaxample: IntStream.range_WithTerminalOperation_0.java, cluster label: 1
For eaxample: IntStream.range_WithTerminalOperation_1.java, cluster label: 1

Processing: java.util.HashMap.get

Total datapoints: 1500 and no of clusters: 284

For eaxample: HashMap.get_CheckKeyNotNull_0.java, cluster label: 2
For eaxample: HashMa

In [ ]:
import pandas as pd
output_path = './API-Minsuse/Repository/Graph-Models/AFGNN/temp/results_mubench_gcb(3.0).xlsx'

df = pd.DataFrame({
    'API name': api_names,
    'Class name': file_names,
    'Cluster Label': labels,
    'Number of Data-Points': num_data_points,
    'Total examples': max_examples,
    'Ground truth label': ground_truth_labels,
    'MR1 (5%)': minimum_required_points[0],
    'MR2 (10%)': minimum_required_points[1],
    'MR3 (15%)': minimum_required_points[2],
    'MR4 (20%)': minimum_required_points[3],
    'MR5 (25%)': minimum_required_points[4],
    'MR6 (30%)': minimum_required_points[5],
    'MR7 (35%)': minimum_required_points[6],
    'MR8 (40%)': minimum_required_points[7],
    'PL1 (5%)': predicted_labels[0],
    'PL2 (10%)': predicted_labels[1],
    'PL3 (15%)': predicted_labels[2],
    'PL4 (20%)': predicted_labels[3],
    'PL5 (25%)': predicted_labels[4],
    'PL6 (30%)': predicted_labels[5],
    'PL7 (35%)': predicted_labels[6],
    'PL8 (40%)': predicted_labels[7]
})

df.to_excel(output_path, index=False)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

output_path = './API-Minsuse/Repository/Graph-Models/AFGNN/temp/evaluation_mubench_gcb(3.0).xlsx'
accuracies, precisions, recalls, f1_scores = [], [], [], []

# Calculate metrics
for i in range(0, len(THRESHOLDS)):
    accuracy = accuracy_score(ground_truth_labels, predicted_labels[i])
    precision = precision_score(ground_truth_labels, predicted_labels[i])
    recall = recall_score(ground_truth_labels, predicted_labels[i])
    f1 = f1_score(ground_truth_labels, predicted_labels[i])
    
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print("================================")
    
df = pd.DataFrame({
    'Threshold': THRESHOLDS,
    'Accuracy': accuracies,
    'Precision': precisions,
    'Recall': recalls,
    'F1 Score': f1_scores
})
df.to_excel(output_path, index=False)

Accuracy: 0.57
Precision: 0.65
Recall: 0.32
F1 Score: 0.43
Accuracy: 0.57
Precision: 0.64
Recall: 0.34
F1 Score: 0.44
Accuracy: 0.53
Precision: 0.54
Recall: 0.39
F1 Score: 0.45
Accuracy: 0.52
Precision: 0.52
Recall: 0.44
F1 Score: 0.48
Accuracy: 0.52
Precision: 0.52
Recall: 0.46
F1 Score: 0.49
Accuracy: 0.52
Precision: 0.52
Recall: 0.48
F1 Score: 0.50
Accuracy: 0.52
Precision: 0.52
Recall: 0.48
F1 Score: 0.50
Accuracy: 0.52
Precision: 0.53
Recall: 0.49
F1 Score: 0.51
